In [1]:
pip install pytrends

In [2]:
pip install pandas

In [9]:
from pytrends.request import TrendReq
import pandas as pd

# Set up pytrends
pytrends = TrendReq(hl='en-US', tz=360)

# Define your list of keywords associated with each stakeholder
keywords = {
    'Businesses' : ['R&D expenditure', 'product development'],
    'VCs' : ['startup funding', 'VC investment'],
    'Financial Institutions' : ['R&D loans', 'R&D financing'],
    'Research Institutions' : ['research grant', 'collaboration with industry'],
    'Government agencies' : ['research funding', 'government grants'],
    'R&D Employees' : ['R&D jobs', 'R&D collaboration tools'],
    'Tax Authorities' : ['R&D tax credit', 'R&D tax incentives'],
    'Consulting Firms' : ['innovation strategy', 'innovation management'],
    'Innovation Hubs' : ['startup incubation', 'technology park'],
    'Patent Attorneys' : ['patent registration', 'patent attorney'],
    'Tax Consultants' : ['tax advice for R&D', 'accounting for R&D']
}

all_suggestions = {}

# Loop through each stakeholder and keyword
for stakeholder, keyword_list in keywords.items():
    for keyword in keyword_list:
        # Fetch suggestions
        suggestions = pytrends.suggestions(keyword=keyword)

        # Convert suggestions to DataFrame and drop 'mid' column
        df = pd.DataFrame(suggestions)
        #.drop(columns='mid')
        df['stakeholder'] = stakeholder
        df['keyword'] = keyword

        # Add to the all_suggestions dictionary
        all_suggestions[f'{stakeholder}_{keyword}'] = df

        print(f'Finished fetching suggestions for {keyword} for {stakeholder}.')

# Combine all suggestions DataFrames
all_suggestions_df = pd.concat(all_suggestions.values(), ignore_index=True)

# Save the DataFrame to a CSV file
all_suggestions_df.to_csv('all_suggestions.csv', index=False)


Finished fetching suggestions for R&D expenditure for Businesses.
Finished fetching suggestions for product development for Businesses.
Finished fetching suggestions for startup funding for VCs.
Finished fetching suggestions for VC investment for VCs.
Finished fetching suggestions for R&D loans for Financial Institutions.
Finished fetching suggestions for R&D financing for Financial Institutions.
Finished fetching suggestions for research grant for Research Institutions.
Finished fetching suggestions for collaboration with industry for Research Institutions.
Finished fetching suggestions for research funding for Government agencies.
Finished fetching suggestions for government grants for Government agencies.
Finished fetching suggestions for R&D jobs for R&D Employees.
Finished fetching suggestions for R&D collaboration tools for R&D Employees.
Finished fetching suggestions for R&D tax credit for Tax Authorities.
Finished fetching suggestions for R&D tax incentives for Tax Authorities.

In [11]:
import time
from pytrends.request import TrendReq
import pandas as pd

# Set up pytrends
pytrends = TrendReq(hl='en-US', tz=360)

# Define your keywords and countries
keywords = {
    'Businesses' : ['R&D expenditure', 'product development'],
    'VCs' : ['startup funding', 'VC investment'],
    'Financial Institutions' : ['R&D loans', 'R&D financing'],
    'Research Institutions' : ['research grant', 'collaboration with industry'],
    'Government agencies' : ['research funding', 'government grants'],
    'R&D Employees' : ['R&D jobs', 'R&D collaboration tools'],
    'Tax Authorities' : ['R&D tax credit', 'R&D tax incentives'],
    'Consulting Firms' : ['innovation strategy', 'innovation management'],
    'Innovation Hubs' : ['startup incubation', 'technology park'],
    'Patent Attorneys' : ['patent registration', 'patent attorney'],
    'Tax Consultants' : ['tax advice for R&D', 'accounting for R&D']
}

countries = ['US', 'CN', 'JP', 'DE', 'KR', 'CA', 'CH', 'GB']

# Initialize an empty dictionary to store the results
results = {}

# Loop over each country, stakeholder, and keyword
for i, country in enumerate(countries, start=1):
    for j, stakeholder in enumerate(keywords, start=1):
        for keyword in keywords[stakeholder]:
            # Fetch Google Trends data
            while True:
                try:
                    pytrends.build_payload([keyword], cat=0, timeframe='all', geo=country, gprop='')
                    trends_data = pytrends.interest_over_time()
                    if not trends_data.empty:
                        trends_data = trends_data.drop(columns=['isPartial'])
                        column_name = f'{country}_{stakeholder}_{keyword}'
                        results[column_name] = trends_data.squeeze()
                    # Print the status
                    print(f'Finished fetching data for {keyword} in {stakeholder} for {country}. ({j}/{len(keywords)} stakeholders, {i}/{len(countries)} countries)')
                    break  # Break the while loop if the request is successful

                except Exception as e:
                    print(f"An error occurred: {e}")
                    print("Sleeping for 60 seconds before retrying...")
                    time.sleep(60)  # Delay for 60 seconds
                    continue  # Retry the failed request

            # Sleep to avoid hitting the rate limit
            time.sleep(10)  # Delay for 10 seconds


print("Fetching data completed.")

# Convert the dictionary to a DataFrame
results_df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
results_df.to_csv('./trends_data.csv')


Finished fetching data for R&D expenditure in Businesses for US. (1/11 stakeholders, 1/8 countries)
An error occurred: The request failed: Google returned a response with code 429
Sleeping for 60 seconds before retrying...
An error occurred: The request failed: Google returned a response with code 429
Sleeping for 60 seconds before retrying...
Finished fetching data for product development in Businesses for US. (1/11 stakeholders, 1/8 countries)
Finished fetching data for startup funding in VCs for US. (2/11 stakeholders, 1/8 countries)
An error occurred: The request failed: Google returned a response with code 429
Sleeping for 60 seconds before retrying...
An error occurred: The request failed: Google returned a response with code 429
Sleeping for 60 seconds before retrying...
An error occurred: The request failed: Google returned a response with code 429
Sleeping for 60 seconds before retrying...
Finished fetching data for VC investment in VCs for US. (2/11 stakeholders, 1/8 countrie